### Limpieza y enriquecimiento de datos en la capa silver.

Se cargan los datos de la capa de bronce a un dataframe para la limpieza básica y enriquecimiento de los datos

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import DoubleType, IntegerType

# Crear la sesión de Spark
spark = SparkSession.builder \
    .appName("stg_covid") \
    .master("local[*]") \
    .getOrCreate()

#  Leer el parquet ingestado de la capa Bronce
df_bronce = spark.read.parquet("/opt/spark/scripts/data/bronce/covid_19_clean_complete")
df_bronce.show(truncate = False)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/22 19:27:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/22 19:27:32 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
                                                                                

+----------------------------+-------------------+---------+----------+----------+---------+------+---------+------+---------------------+
|Province/State              |Country/Region     |Lat      |Long      |Date      |Confirmed|Deaths|Recovered|Active|WHO Region           |
+----------------------------+-------------------+---------+----------+----------+---------+------+---------+------+---------------------+
|NULL                        |Afghanistan        |33.93911 |67.709953 |2020-01-22|0        |0     |0        |0     |Eastern Mediterranean|
|NULL                        |Albania            |41.1533  |20.1683   |2020-01-22|0        |0     |0        |0     |Europe               |
|NULL                        |Algeria            |28.0339  |1.6596    |2020-01-22|0        |0     |0        |0     |Africa               |
|NULL                        |Andorra            |42.5063  |1.5218    |2020-01-22|0        |0     |0        |0     |Europe               |
|NULL                      

Se renombran columnas

In [2]:
df_silver = df_bronce.select(
    col("Province/State").alias("provincia_estado"),
    col("Country/Region").alias("pais"),
    col("Lat").cast(DoubleType()).alias("lat"),
    col("Long").cast(DoubleType()).alias("long"),
    to_date(col("Date"), "yyyy-MM-dd").alias("fecha"),
    col("Confirmed").cast(IntegerType()).alias("confirmados"),
    col("Deaths").cast(IntegerType()).alias("muertes"),
    col("Recovered").cast(IntegerType()).alias("recuperados"),
    col("Active").cast(IntegerType()).alias("activos"),
    col("WHO Region").alias("region_who")
)

Se limpian registros inválidos (por ejemplo, sin país o fecha)

In [4]:
df_silver = df_silver.filter(col("pais").isNotNull() & col("fecha").isNotNull())

Se quitan registros que tengan todos los valores métricos en null (si existieran)

In [3]:
df_silver = df_silver.filter(
    col("confirmados").isNotNull() |
    col("muertes").isNotNull() |
    col("recuperados").isNotNull() |
    col("activos").isNotNull()
)

Se reemplazan nulls por 0 en métricas numéricas y con el texto desconocido en el caso de la "provincia_estado"

In [4]:
df_silver = df_silver.fillna({
    "confirmados": 0,
    "muertes": 0,
    "recuperados": 0,
    "activos": 0,
    "provincia_estado": "Desconocido"
})

Guardar con formato .parquet en la capa Silver

In [5]:
df_silver.write.mode("overwrite").parquet("/opt/spark/scripts/data/silver/casos_diarios")

Se realiza la consulta para ver el resultado en la capa silver

In [7]:
df = spark.read.parquet("/opt/spark/scripts/data/silver/casos_diarios")
df.show()

+--------------------+-------------------+---------+----------+----------+-----------+-------+-----------+-------+--------------------+
|    provincia_estado|               pais|      lat|      long|     fecha|confirmados|muertes|recuperados|activos|          region_who|
+--------------------+-------------------+---------+----------+----------+-----------+-------+-----------+-------+--------------------+
|         Desconocido|        Afghanistan| 33.93911| 67.709953|2020-01-22|          0|      0|          0|      0|Eastern Mediterra...|
|         Desconocido|            Albania|  41.1533|   20.1683|2020-01-22|          0|      0|          0|      0|              Europe|
|         Desconocido|            Algeria|  28.0339|    1.6596|2020-01-22|          0|      0|          0|      0|              Africa|
|         Desconocido|            Andorra|  42.5063|    1.5218|2020-01-22|          0|      0|          0|      0|              Europe|
|         Desconocido|             Angola| -11.2